In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
firstDS = pd.read_csv('/content/drive/MyDrive/final_updated_555_with_realistic_pilot_replies.csv')
secondDS = pd.read_csv("/content/drive/MyDrive/enhanced_555lableBased.csv")
thirdDS = pd.read_csv("/content/drive/MyDrive/updated_555_with_pilot_replies.csv")
fourthDS = pd.read_csv("/content/drive/MyDrive/full_updated_pilot_communications.csv")
fifthDS = pd.read_csv("/content/drive/MyDrive/generalized_pilot_communications.csv")
sixthDS = pd.read_csv("/content/drive/MyDrive/unique_pilot_communications.csv")
seventhDS = pd.read_csv("/content/drive/MyDrive/robust_unique_replies_pilot_communications.csv")
eighthDS = pd.read_csv("/content/drive/MyDrive/updated_atc_pilot_communications.csv")

In [3]:
firstDS_labels = firstDS["text"]
secondDS_labels = secondDS["text"]
thirdDS_labels = thirdDS["text"]
firstDS_replies = firstDS["Pilot Reply"]
secondDS_replies = secondDS["Pilot Reply"]
thirdDS_replies = thirdDS["Pilot Reply"]

In [4]:
firstDS_labels = firstDS["text"]
secondDS_labels = secondDS["text"]
thirdDS_labels = thirdDS["text"]
firstDS_replies = firstDS["Pilot Reply"]
secondDS_replies = secondDS["Pilot Reply"]
thirdDS_replies = thirdDS["Pilot Reply"]
fourthDS_labels = fourthDS["text"]
fourthDS_replies = fourthDS["Pilot Reply"]
fifthDS_labels = fifthDS["text"]
fifthDS_replies = fifthDS["Pilot Reply"]
sixthDS_labels = sixthDS["text"]
sixthDS_replies = sixthDS["Pilot Reply"]
seventhDS_labels = seventhDS["text"]
seventhDS_replies = seventhDS["Pilot Reply"]
eighthDS_labels = eighthDS["text"]
eighthDS_replies = eighthDS["Pilot Reply"]

In [5]:
labels = []
for row in range(10077):
    labels.append(f"{firstDS_labels[row]}")
for row in range(10077):
    labels.append(f"{secondDS_labels[row]}")
for row in range(10077):
    labels.append(f"{thirdDS_labels[row]}")
for row in range(10077):
    labels.append(f"{fourthDS_labels[row]}")
replies = []
for row in range(10077):
    replies.append(f"{firstDS_replies[row]}")
for row in range(10077):
    replies.append(f"{secondDS_replies[row]}")
for row in range(10077):
    replies.append(f"{thirdDS_replies[row]}")
for row in range(10077):
    replies.append(f"{fourthDS_replies[row]}")
for row in range(10077):
    labels.append(f"{fifthDS_labels[row]}")
for row in range(10077):
    replies.append(f"{fifthDS_replies[row]}")
for row in range(10077):
    labels.append(f"{sixthDS_labels[row]}")
for row in range(10077):
    replies.append(f"{sixthDS_replies[row]}")
for row in range(335):
    labels.append(f"{seventhDS_labels[row]}")
for row in range(335):
    replies.append(f"{seventhDS_replies[row]}")
for row in range(127):
    labels.append(f"{eighthDS_labels[row]}")
for row in range(127):
    replies.append(f"{eighthDS_replies[row]}")

In [6]:
labels

['gulf air zero zero seven radar contact ',
 'good morning speedbird one two four radar contact ',
 'lufthansa eight two six eight good morning climb to flight level two nine zero ',
 'speedbird one two four contact rhein on one two seven three seven good day ',
 'belstar five zero two hotel rhein on one three two four ',
 'alitalia four one three continue climb to level two nine zero ',
 'ah good morning netherlands air force three five in radar contact ',
 'viva five eight five rhein ',
 'fox sierra india guten tag climb to level two seven zero ',
 'sabena eight zero one direct trasadingen ',
 'sabena eight three two call rhein on one three correction one two seven three seven ',
 'swissair six five two zero guten tag in radar contact ',
 'viva five eight five report your heading ',
 'air malta zero zero four direct to frankfurt ',
 'good bye ',
 'alitalia four eight seven zurich one three four six ',
 'sobelair two five five seven call rhein on one three two decimal four ',
 'aero l

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(labels + replies)

In [9]:
input_sequences = tokenizer.texts_to_sequences(labels)

In [10]:
sequences = tokenizer.texts_to_sequences(replies)

In [11]:
max_sequence_len = max([len(x) for x in sequences])

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
input_sequences_padded = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='post')
output_sequences_padded = pad_sequences(sequences, maxlen=max_sequence_len, padding='post')

In [14]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [15]:
model = Sequential([Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=10),
                    LSTM(128, return_sequences=True),
                    Dense(len(tokenizer.word_index) + 1, activation='softmax') ])

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(input_sequences_padded, output_sequences_padded, epochs=3,shuffle=True, batch_size=64)

Epoch 1/3
952/952 ━━━━━━━━━━━━━━━━━━━━ 88s 89ms/step - accuracy: 0.6657 - loss: 2.4428
Epoch 2/3
952/952 ━━━━━━━━━━━━━━━━━━━━ 147s 95ms/step - accuracy: 0.6810 - loss: 1.4486
Epoch 3/3
952/952 ━━━━━━━━━━━━━━━━━━━━ 139s 92ms/step - accuracy: 0.6817 - loss: 1.3161


In [18]:
def generate_reply(model, tokenizer, label, temperature=1.0):
    sequence = tokenizer.texts_to_sequences([label])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_len, padding='post')
    prediction = model.predict(padded_sequence)
    prediction = prediction.astype('float64')
    prediction = np.log(prediction + 1e-7) / temperature
    prediction = np.exp(prediction)
    prediction /= np.sum(prediction, axis=-1, keepdims=True)
    predicted_word_indices = []
    for prob in prediction[0]:
        predicted_word_indices.append(np.random.choice(len(prob), p=prob))
    reply = ' '.join(tokenizer.index_word.get(i, '') for i in predicted_word_indices if i != 0)
    return reply

In [20]:
import re
def airport_first(outtext,intext,airportLst):
    for airport_name in airportLst:
        name_pattern = f"\b{airport_name}\b"
        name_pattern = f"{airport_name}"
        #print(name_pattern)
        match = re.findall(name_pattern, intext)
        matchout = re.findall(name_pattern, outtext)
        if match:
            airport_name_found = match[0]
            cleaned_text = re.sub(name_pattern, '', outtext).strip()
            return f"{airport_name_found}, {cleaned_text}"
            break
        elif matchout:
            airport_name_found = matchout[0]
            cleaned_text = re.sub(name_pattern, '', outtext).strip()
            return f"{airport_name_found}, {cleaned_text}"
            break
        else:
            if airport_name == airportLst[-1]:
                return outtext
                break
            else:
                continue

In [21]:
def name_of_flight(outp2, inp , numbers):
    num_pattern = r"(?:(?:zero|one|two|three|four|five|six|seven|eight|nine|ten)\s)+"
    match = re.findall(num_pattern, inp)
        #string = string + str(match)
    for number in numbers:
        n_p = f"{number}"
        if re.findall(n_p, outp2):
                cleaned_text = re.sub(num_pattern, '', outp2).strip()
    #print(match)
    if match:
        splitted = outp2.split(",")
        #print(splitted)
        added = f"{splitted[0]} {match[0]} {splitted[1]}"
    else:
        added = outp2
    cleaned_text = re.sub(r'\s+', ' ', added)
    return cleaned_text

In [22]:
def strip_ending(input):
    ending_pattern = r"(?:\s(?:for|by|(?:for\sthe)|to|at|the|and|out|of|as|thank)$)|(?:\s(?:for|by|(?:for\sthe)|to|at|the|and|out|of|as|thank)\s$)"
    clear_text = input
    for y in range(7):
        #print(clear_text)
        if re.findall(ending_pattern, input):
            clear_text = re.sub(ending_pattern, '', clear_text).strip()
            #print(clear_text)
        else:
            break
    return clear_text

In [23]:
clean_airport_names = airline_names = [
    "lufthansa", "sabena", "transwede", "belstar", "india oscar", "transavia", "speedbird", "swissair",
    "foxtrot sierra", "india tango", "india sierra", "hapag lloyd", "malaysian", "constellation", "olympic",
    "georgia air","alitalia", "sobelair", "buongiorno alitalia", "jet set", "air malta", "viva", "aero lloyd", "delta india",
    "britannia", "belstar", "gulf air", "netherlands air", "turkish", "europa", "midland", "condor",
    "emirates", "delta echo", "saudia", "scandinavian", "cross air", "merair", "meridiana", "adria",
    "egyptair", "tunisair", "sunwing", "air algerie", "tarom", "air berlin", "hotel bravo", "algerie",
    "leisure", "air georgia", "turkish", "turkair", "delta papa", "london airtours", "belgian airforce",
    "german air", "onur air", "finnair", "hamburg air", "air portugal", "portugalia", "corsair", "sata",
    "monarch", "air inter", "shamrock", "futura", "condor", "gibair", "skyfox", "air europa", "palestine air",
    "lauda air", "air link"]
numbers = ["one","two","three","four","five","six","seven","eight","nine"]
not_ending = ["for", "by", "to", "and", "out", "of", "for the", "as"]

In [24]:
import time
def generate_total_reply(inp):
    outp = generate_reply(model, tokenizer, inp )
    outp2 = airport_first(outp,inp,clean_airport_names)
    outp3 = name_of_flight(outp2,inp,numbers)
    outp4 = strip_ending(outp3)
    time.sleep(0.5)
    return outp4
def measure_metrics(input_text, num_requests=10):
    response_times = []
    error_count = 0

    for _ in range(num_requests):
        start_time = time.time()
        try:
            response = generate_total_reply(input_text)
            if not response:
                error_count += 1
        except Exception as e:
            print(f"Request failed with exception: {e}")
            error_count += 1
        end_time = time.time()

        response_times.append(end_time - start_time)

    total_requests = num_requests
    successful_requests = total_requests - error_count
    throughput = successful_requests / sum(response_times) if sum(response_times) > 0 else 0
    average_latency = sum(response_times) / successful_requests if successful_requests > 0 else float('inf')
    percentile_90 = sorted(response_times)[int(0.9 * len(response_times))] if response_times else float('inf')
    percentile_99 = sorted(response_times)[int(0.99 * len(response_times))] if response_times else float('inf')
    error_rate = error_count / total_requests

    metrics = {
        "total_requests": total_requests,
        "successful_requests": successful_requests,
        "error_count": error_count,
        "error_rate": error_rate,
        "average_latency": average_latency,
        "throughput": throughput,
        "90th_percentile_latency": percentile_90,
        "99th_percentile_latency": percentile_99,
        "response_times": response_times
    }

    return metrics

def main():
    input_text = "hapag lloyd one two four climb to flight level three six zero "
    num_requests = 100

    metrics = measure_metrics(input_text, num_requests)

    print(f"Metrics: {metrics}")

if __name__ == "__main__":
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━

In [25]:
generate_total_reply("alitalia one four nine for shortcut right one five degrees")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


'lufthansa one four nine received thank you communication initiating for additional'